# Setup

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import re

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim

from PIL import Image
from io import BytesIO
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Sampling DataSet

In [ ]:
# Read the dataset from Hugging Face using Dask
df = dd.read_parquet(
    "hf://datasets/raghavendrad60/vqa_plant-disease-classification-merged-dataset/data/train-*.parquet",
    columns=['class', 'image']
)

# Plants to keep
patterns = ['Tomato', 'Chili', 'Cucumber', 'Strawberry', 'Pepper']
desired_plants = [p.lower() for p in patterns]

# Filter function to include only desired plants
def filter_partition(df):
    df['class'] = df['class'].astype(str).str.strip().str.lower()
    df['plant'] = df['class'].str.extract(r'^([^_]+)')
    return df[df['plant'].isin(desired_plants)]

df = df.map_partitions(filter_partition).persist()

# Function to sample a small number of rows from a partition
def sample_partition(partition, n_per_partition=400):
    return partition.sample(n=min(len(partition), n_per_partition), random_state=np.random.randint(0, 1e6))

# Sample a limited number of rows from each partition
sampled = df.map_partitions(sample_partition, n_per_partition=400)

# Collect the sampled data into memory, then take a final random sample of 50000
sampled_df = sampled.compute()
final_sample = sampled_df.sample(n=min(50000, len(sampled_df)), random_state=42)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
final_sample

,class,image,plant
1665,tomato__mosaic_virus,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,tomato
674,strawberry___leaf_scorch,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,strawberry
2005,strawberry__healthy,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,strawberry
1722,tomato__healthy,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,tomato
1147,tomato__septoria_leaf_spot,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,tomato
...,...,...,...
1638,tomato__mosaic_virus,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,tomato
43,cucumber__healthy,{'bytes': b'\xff\xd8\xff\xe1\xa3\x90Exif\x00\x...,cucumber
98,cucumber__healthy,{'bytes': b'\xff\xd8\xff\xe1\xa3\x90Exif\x00\x...,cucumber
119,cucumber__healthy,{'bytes': b'\xff\xd8\xff\xe1\xa3\x90Exif\x00\x...,cucumber


# Training

In [ ]:
# Count samples per class
class_counts = final_sample['class'].value_counts()
print(class_counts)

# Filter out classes with fewer than 2 samples
valid_classes = class_counts[class_counts >= 2].index
final_sample = final_sample[final_sample['class'].isin(valid_classes)]

class
tomato__yellow_leaf_curl_virus                    752
tomato__target_spot                               729
tomato__late_blight                               577
tomato__septoria_leaf_spot                        469
tomato__leaf_mold                                 413
tomato__spider_mites_(two_spotted_spider_mite)    400
strawberry___leaf_scorch                          400
strawberry__healthy                               387
tomato__healthy                                   381
cucumber__healthy                                 341
tomato__bacterial_spot                            331
pepper_bell__healthy                              328
tomato__mosaic_virus                              287
tomato__early_blight                              266
pepper_bell__bacterial_spot                       206
chili__yellowish                                  100
chili__leaf curl                                  100
cucumber__diseased                                 82
chili__healthy        

In [ ]:
# Extract actual byte data from the dictionary in the 'image' column
final_sample['image_bytes'] = final_sample['image'].apply(lambda x: eval(x)['bytes'] if isinstance(x, str) else x['bytes'])

# Encode class labels
le = LabelEncoder()
final_sample['label'] = le.fit_transform(final_sample['class'])

# Split
train_df, val_df = train_test_split(final_sample, test_size=0.2, stratify=final_sample['label'], random_state=42)

# Define transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [ ]:
class ByteImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_bytes = self.df.iloc[idx]['image_bytes']
        image = Image.open(BytesIO(img_bytes)).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.df.iloc[idx]['label']
        return image, label

train_dataset = ByteImageDataset(train_df, transform)
val_dataset = ByteImageDataset(val_df, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, final_sample['label'].nunique())
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(8):  # Adjust as needed
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Evaluation
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 135MB/s]


Epoch 1, Loss: 116.0581
Epoch 2, Loss: 57.1833
Epoch 3, Loss: 40.5751


# Inference (Prediction)

In [ ]:
# Same image preprocessing used during training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Inference function
def predict_from_bytes(img_bytes):
    model.eval()
    image = Image.open(BytesIO(img_bytes)).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        class_index = predicted.item()
        return le.inverse_transform([class_index])[0]  # Return class label

In [ ]:
# Picture input (from file or database)
with open("/content/Tomato_Leaf_Mold_Test3", "rb") as f:
    img_bytes = f.read()

predicted_class = predict_from_bytes(img_bytes)
print("Predicted class:", predicted_class)

#Save the Model using Pickle

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pickle

# Encode labels
le = LabelEncoder()
final_sample['label'] = le.fit_transform(final_sample['class'])

# Save label encoder
with open('label_encoder_vF.pkl', 'wb') as f:
    pickle.dump(le, f)

# Save label encoder
model_info = {
    'state_dict': model.state_dict(),
    'num_classes': final_sample['label'].nunique()
}

with open('advanced_ml_project_model_vF.pkl', 'wb') as f:
    pickle.dump(model_info, f)